# 1. package import

In [1]:
from jbfg.pandas_utils import *
import pickle
import pandas as pd

# 2. dataset 불러오기
 - 과거 구매 이력 및 구매 상품 정보

In [2]:
schema = 'l2_kjbk_extnl'

In [3]:
sql1 = """
       SELECT *
       FROM   train_x_past
       """

In [4]:
sql2 = """
       SELECT *
       FROM   products_info
       """

In [5]:
df1 = get_df(sql1, schema)
df2 = get_df(sql2, schema)

In [6]:
display(df1)
display(df2)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,717029,196613,15,1,9,3.0,5,0,41220
1,2325772,196613,35,6,11,2.0,7,1,41220
2,642668,196613,69,2,16,2.0,11,1,4210
3,1326515,196613,61,3,19,3.0,3,1,4210
4,1437845,196613,36,3,15,4.0,16,0,4210
...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852
1253968,1969590,163819,42,1,15,6.0,3,1,24852
1253969,2549215,163819,9,1,11,7.0,1,1,24852
1253970,3103211,163819,39,1,14,7.0,3,1,24852


,product_id,name,category_id,division_id,category_name,division_name
0,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
1,20995,Organic Broccoli Florets,116,1,frozen produce,frozen
2,33000,Pure Irish Butter,36,16,butter,dairy eggs
3,7781,Organic Sticks Low Moisture Part Skim Mozzarel...,21,16,packaged cheese,dairy eggs
4,47144,Unsweetened Original Almond Breeze Almond Milk,91,16,soy lactosefree,dairy eggs
...,...,...,...,...,...,...
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce


In [7]:
df = pd.merge(df1, df2, how='left', on='product_id')

In [8]:
df

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
0,717029,196613,15,1,9,3.0,5,0,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
1,2325772,196613,35,6,11,2.0,7,1,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
2,642668,196613,69,2,16,2.0,11,1,4210,Whole Milk,84,16,milk,dairy eggs
3,1326515,196613,61,3,19,3.0,3,1,4210,Whole Milk,84,16,milk,dairy eggs
4,1437845,196613,36,3,15,4.0,16,0,4210,Whole Milk,84,16,milk,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852,Banana,24,4,fresh fruits,produce
1253968,1969590,163819,42,1,15,6.0,3,1,24852,Banana,24,4,fresh fruits,produce
1253969,2549215,163819,9,1,11,7.0,1,1,24852,Banana,24,4,fresh fruits,produce
1253970,3103211,163819,39,1,14,7.0,3,1,24852,Banana,24,4,fresh fruits,produce


# 3. New columns make
 - 평일, 주말 여부
 - 오전, 오후 여부
 - 아침, 점심, 저녁시간 여부
 - 주문순서 구간
 - 고객별 총 주문 횟수
 - 고객별 총 주문 상품 갯수(unique)
 - 상품 품목별 구매 경험
 - 

In [9]:
def weekday_yn(week_day):
    if week_day in ('1','2','3','4','5') : # 월화수목금
        return 1
    else:
        return 0

In [10]:
def weekend_yn(week_day):
    if week_day in ('6','0') : # 토일
        return 1
    else:
        return 0

In [11]:
def morning_yn(order_hour):
    if order_hour in ('0','1','2','3','4','5','6','7','8','9','10','11') : # 오전
        return 1
    else:
        return 0

In [12]:
def afternoon_yn(order_hour):
    if order_hour in ('12','13','14','15','16','17','18','19','20','21','22','23') : # 오후
        return 1
    else:
        return 0

In [13]:
def breakfast_yn(order_hour):
    if order_hour in ('6','7','8','9') :    # 아침
        return 1
    else:
        return 0

In [14]:
def lunch_yn(order_hour):
    if order_hour in ('11','12','13','14') : # 점심
        return 1
    else:
        return 0

In [15]:
def dinner_yn(order_hour):
    if order_hour in ('17','18','19','20') : # 저녁
        return 1
    else:
        return 0

In [16]:
def order_nums(add_order):
    if add_order == '1':
        return 1
    if add_order in ('2','3','4','5'):
        return 2
    if add_order in ('6','7','8','9','10'):
        return 3
    if add_order in ('11','12','13','14','15','16','17','18','19','20'):
        return 4
    if add_order in ('21','22','23','24','25','26','27','28','29','30'):
        return 5
    if add_order in ('31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50'):
        return 6
    if add_order in ('51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80'):
        return 7
    if add_order in ('81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99','100'):
        return 8
    else:
        return 0

In [17]:
df['weekday_yn']   = df.apply(lambda x: weekday_yn(x.week_day)    , axis=1)
df['weekend_yn']   = df.apply(lambda x: weekend_yn(x.week_day)    , axis=1)
df['morning_yn']   = df.apply(lambda x: morning_yn(x.order_hour)  , axis=1)
df['afternoon_yn'] = df.apply(lambda x: afternoon_yn(x.order_hour), axis=1)
df['breakfast_yn'] = df.apply(lambda x: breakfast_yn(x.order_hour), axis=1)
df['lunch_yn']     = df.apply(lambda x: lunch_yn(x.order_hour)    , axis=1)
df['dinner_yn']    = df.apply(lambda x: dinner_yn(x.order_hour)   , axis=1)
df['order_nums']   = df.apply(lambda x: order_nums(x.add_order)   , axis=1)

In [18]:
order_cnt    = pd.DataFrame(df.groupby(['cstno']).order_id.nunique()).rename(columns={'order_id':'order_cnt'})
product_cnt  = pd.DataFrame(df.groupby(['cstno']).product_id.nunique()).rename(columns={'product_id':'product_cnt'})
division_hst = pd.DataFrame(df.groupby(['cstno']).division_name.nunique()).rename(columns={'division_name':'division_hst'})
category_hst = pd.DataFrame(df.groupby(['cstno']).category_name.nunique()).rename(columns={'category_name':'category_hst'})

In [19]:
df = pd.merge(df, order_cnt   , how='left', on='cstno')
df = pd.merge(df, product_cnt , how='left', on='cstno')
df = pd.merge(df, division_hst, how='left', on='cstno')
df = pd.merge(df, category_hst, how='left', on='cstno')

In [20]:
df

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,morning_yn,afternoon_yn,breakfast_yn,lunch_yn,dinner_yn,order_nums,order_cnt,product_cnt,division_hst,category_hst
0,717029,196613,15,1,9,3.0,5,0,41220,Organic Romaine Lettuce,...,1,0,1,0,0,2,70,42,4,8
1,2325772,196613,35,6,11,2.0,7,1,41220,Organic Romaine Lettuce,...,1,0,0,1,0,3,70,42,4,8
2,642668,196613,69,2,16,2.0,11,1,4210,Whole Milk,...,0,1,0,0,0,4,70,42,4,8
3,1326515,196613,61,3,19,3.0,3,1,4210,Whole Milk,...,0,1,0,0,1,2,70,42,4,8
4,1437845,196613,36,3,15,4.0,16,0,4210,Whole Milk,...,0,1,0,0,0,4,70,42,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852,Banana,...,0,1,0,0,1,2,47,20,5,9
1253968,1969590,163819,42,1,15,6.0,3,1,24852,Banana,...,0,1,0,0,0,2,47,20,5,9
1253969,2549215,163819,9,1,11,7.0,1,1,24852,Banana,...,1,0,0,1,0,1,47,20,5,9
1253970,3103211,163819,39,1,14,7.0,3,1,24852,Banana,...,0,1,0,1,0,2,47,20,5,9


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253972 entries, 0 to 1253971
Data columns (total 26 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   order_id        1253972 non-null  object
 1   cstno           1253972 non-null  object
 2   order_number    1253972 non-null  object
 3   week_day        1253972 non-null  object
 4   order_hour      1253972 non-null  object
 5   order_interval  1253972 non-null  object
 6   add_order       1253972 non-null  object
 7   is_reordered    1253972 non-null  object
 8   product_id      1253972 non-null  object
 9   name            1253972 non-null  object
 10  category_id     1253972 non-null  object
 11  division_id     1253972 non-null  object
 12  category_name   1253972 non-null  object
 13  division_name   1253972 non-null  object
 14  weekday_yn      1253972 non-null  int64 
 15  weekend_yn      1253972 non-null  int64 
 16  morning_yn      1253972 non-null  int64 
 17  afternoo

# 4. 데이터 전처리
 - 문자형 -> 수치형으로 변경(order_interval: 직전구매후경과일수)
 - 범주형 데이터 onehotencoding(week_day:구매요일 , is_reordered: 재구매여부)
 - columns merge

In [23]:
df['order_interval'] = pd.to_numeric(df['order_interval'])

In [24]:
cate_cols = set(['week_day','is_reordered'])
col       = set(df.columns.to_list())
normal_cols = list(col-cate_cols)
cate_cols   = list(cate_cols)

In [25]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values='null', strategy='constant', fill_value='missing')

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [26]:
df_pp1 = imputer.fit_transform(df[cate_cols])
df_pp2 = encoder.fit_transform(df_pp1)
df_pp3 = pd.DataFrame(df_pp2, columns=encoder.get_feature_names(cate_cols))

In [27]:
dfF = pd.merge(df, df_pp3, how='left', left_index=True, right_index=True)

In [28]:
dfF

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,category_hst,is_reordered_0,is_reordered_1,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6
0,717029,196613,15,1,9,3.0,5,0,41220,Organic Romaine Lettuce,...,8,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2325772,196613,35,6,11,2.0,7,1,41220,Organic Romaine Lettuce,...,8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,642668,196613,69,2,16,2.0,11,1,4210,Whole Milk,...,8,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1326515,196613,61,3,19,3.0,3,1,4210,Whole Milk,...,8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1437845,196613,36,3,15,4.0,16,0,4210,Whole Milk,...,8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852,Banana,...,9,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1253968,1969590,163819,42,1,15,6.0,3,1,24852,Banana,...,9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1253969,2549215,163819,9,1,11,7.0,1,1,24852,Banana,...,9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1253970,3103211,163819,39,1,14,7.0,3,1,24852,Banana,...,9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
dfF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253972 entries, 0 to 1253971
Data columns (total 35 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   order_id        1253972 non-null  object 
 1   cstno           1253972 non-null  object 
 2   order_number    1253972 non-null  object 
 3   week_day        1253972 non-null  object 
 4   order_hour      1253972 non-null  object 
 5   order_interval  1232286 non-null  float64
 6   add_order       1253972 non-null  object 
 7   is_reordered    1253972 non-null  object 
 8   product_id      1253972 non-null  object 
 9   name            1253972 non-null  object 
 10  category_id     1253972 non-null  object 
 11  division_id     1253972 non-null  object 
 12  category_name   1253972 non-null  object 
 13  division_name   1253972 non-null  object 
 14  weekday_yn      1253972 non-null  int64  
 15  weekend_yn      1253972 non-null  int64  
 16  morning_yn      1253972 non-null  in

# 5. Train df 생성

In [45]:
# 제외 컬럼(키값 등) 
col_except = ['order_id'    ,'cstno'   ,'category_id','category_name','division_id' ,'division_name','name',
              'order_number','week_day','order_hour' ,'add_order'    ,'is_reordered','product_id'   ]

In [46]:
Train_x = dfF[list(dfF.columns.difference(col_except))]
Train_y = dfF['product_id']

In [47]:
display(Train_x)
display(Train_y)

,afternoon_yn,breakfast_yn,category_hst,dinner_yn,division_hst,is_reordered_0,is_reordered_1,lunch_yn,morning_yn,order_cnt,...,product_cnt,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4,week_day_5,week_day_6,weekday_yn,weekend_yn
0,0,1,8,0,4,1.0,0.0,0,1,70,...,42,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0,0,8,0,4,0.0,1.0,1,1,70,...,42,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1
2,1,0,8,0,4,0.0,1.0,0,0,70,...,42,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0
3,1,0,8,1,4,0.0,1.0,0,0,70,...,42,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
4,1,0,8,0,4,1.0,0.0,0,0,70,...,42,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1,0,9,1,5,0.0,1.0,0,0,47,...,20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1253968,1,0,9,0,5,0.0,1.0,0,0,47,...,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
1253969,0,0,9,0,5,0.0,1.0,1,1,47,...,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
1253970,1,0,9,0,5,0.0,1.0,1,0,47,...,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0


0          41220
1          41220
2           4210
3           4210
4           4210
           ...  
1253967    24852
1253968    24852
1253969    24852
1253970    24852
1253971    24852
Name: product_id, Length: 1253972, dtype: object

# 6. Model fit(LGBM)

In [52]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
from numba import jit

In [49]:
md_lgbm = LGBMClassifier(random_state = 1234)

In [51]:
%%time
md_lgbm.fit(Train_x, Train_y)

CPU times: user 5h 28min 59s, sys: 30min 16s, total: 5h 59min 16s
Wall time: 8min 55s


LGBMClassifier(random_state=1234)